<a href="https://colab.research.google.com/github/LCaselles/TP_NLP/blob/main/TP_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP : Moteur de recherche basé sur le modèle TF-IDF**


Ce TP a pour objectif de construire un moteur de recherche en utilisant la technique **TF-IDF** (Term Frequency - Inverse Document Frequency). Le TF-IDF est une méthode couramment utilisée en traitement automatique des langues (TAL) pour évaluer l'importance d'un mot dans un document par rapport à un ensemble de documents. Le but est de permettre aux utilisateurs de rechercher des documents en fonction d'une requête textuelle et de retourner les documents les plus pertinents.

---

## Outils et bibliothèques utilisés

Pour ce TP, nous utiliserons les outils et bibliothèques suivants :
- **Python** : Langage principal pour le traitement des données.
- **pandas** : Pour la manipulation des données sous forme de tableaux (DataFrames).
- **spaCy** : Pour effectuer le prétraitement linguistique, notamment la lemmatisation et la gestion des stopwords.
- **scikit-learn** : Pour calculer le TF-IDF avec `TfidfVectorizer` et la similarité cosinus entre les documents et la requête.

Le dataset utilisé est un ensemble de discours extraits des comptes rendus de débats de l'Assemblée Nationale, stockés dans un fichier CSV, qui servira de base pour la construction du moteur de recherche.

---

## **Partie 1 : Lecture des données**

### Objectif

L'objectif de cette première partie est de charger le dataset contenant les discours extraits des débats de l'Assemblée Nationale depuis un fichier CSV. Ce dataset servira de base pour le calcul du TF-IDF et la construction du moteur de recherche.

---

### Consignes

1. Charger le fichier CSV contenant les documents en utilisant la bibliothèque `pandas`.
2. Afficher les premières lignes du DataFrame pour visualiser les données.

---

In [5]:
import pandas as pd

# Charger les documents depuis le fichier CSV
df = pd.read_csv('speeches_debates.csv')

# Afficher les premières lignes pour vérifier le contenu
df.head()

,Document
0,Conformément au premier alinéa de l’article 28...
1,Philippine avait 19 ans seulement. Le 21 septe...
2,Je suis heureuse de souhaiter en votre nom à t...
3,L’ordre du jour appelle la déclaration du Gouv...
4,"Mesdames, messieurs les députés, avant de comm..."


In [6]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 60.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## **Partie 2 : Prétraitement des documents**

### Objectif

L'objectif de cette partie est d'effectuer un prétraitement des documents afin d'uniformiser le texte et d'améliorer les résultats lors du calcul du TF-IDF. Le prétraitement consiste à :

---

### Consignes

1. Transformer tous les documents en minuscules pour éviter que les mots "Apple" et "apple" soient considérés comme différents.
2. Supprimer toute la ponctuation, afin de ne pas inclure des signes inutiles dans le calcul du TF-IDF.
3. Utiliser le modèle de langue française de `spaCy` pour effectuer la lemmatisation des documents.
4. Supprimer les stopwords, ces mots courants qui n'apportent pas de valeur ajoutée à la recherche.

---

In [10]:
import spacy
import re
import string

# Charger le modèle français de spaCy pour la lemmatisation
nlp = spacy.load("fr_core_news_sm")
# Fonction de prétraitement
# Fonction de prétraitement
def pretraiter_texte(texte):
    # Transformer en minuscules
    texte = texte.lower()

    # Supprimer la ponctuation
    texte = texte.translate(str.maketrans('', '', string.punctuation))

    # Lemmatisation avec spaCy
    doc = nlp(texte)
    texte_lemmatise = ' '.join([token.lemma_ for token in doc])

    return texte_lemmatise

# Appliquer le prétraitement à tous les documents
df['Document_pretraite'] = df['Document'].apply(pretraiter_texte)

# Vérifier le résultat du prétraitement
print(df['Document_pretraite'].head())

0    conformément au premier alinéa de l’ article  ...
1    philippin avoir 19   an seulement le 21   sept...
2    je être heureux de souhaiter en votre nom à to...
3    l’ ordre de jour appeler le déclaration de gou...
4    mesdame monsieur le député avant de commencer ...
Name: Document_pretraite, dtype: object


## **Partie 3 : Calcul du TF-IDF**

### Objectif

L'objectif de cette partie est de calculer la matrice **TF-IDF** (Term Frequency - Inverse Document Frequency) pour les documents prétraités. Le TF-IDF permet de quantifier l'importance d'un mot dans un document donné par rapport à un ensemble de documents. Il est utilisé pour évaluer la pertinence d'un mot dans un moteur de recherche.

---

### Consignes

1. Utiliser la fonction `TfidfVectorizer` de la bibliothèque `scikit-learn` pour calculer le TF-IDF des documents prétraités.
2. Calculer la matrice TF-IDF pour l'ensemble des documents.
3. Observer la forme de la matrice résultante (nombre de documents vs nombre de termes).
4. Visualiser le vecteur correspondant au premier document.


---

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisation du vectorizer
vectorizer = TfidfVectorizer()

# Calcul de la matrice TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['Document_pretraite'])

# Afficher la forme de la matrice TF-IDF (nombre de documents, nombre de termes)
print(f"Forme de la matrice TF-IDF : {tfidf_matrix.shape}")

# Visualiser le vecteur correspondant au premier document
first_doc_vector = tfidf_matrix[0]  # Vecteur du premier document
print(first_doc_vector)

Forme de la matrice TF-IDF : (303, 2223)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15 stored elements and shape (1, 2223)>
  Coords	Values
  (0, 445)	0.3025823031041817
  (0, 215)	0.1337188440095265
  (0, 1546)	0.15794932056468242
  (0, 109)	0.3244188098818042
  (0, 547)	0.12538282977705356
  (0, 177)	0.287089054981786
  (0, 25)	0.3025823031041817
  (0, 1134)	0.12249938289713172
  (0, 469)	0.27507155810421097
  (0, 1103)	0.12575253850413812
  (0, 646)	0.287089054981786
  (0, 1412)	0.2569506936240523
  (0, 1883)	0.3244188098818042
  (0, 1394)	0.3244188098818042
  (0, 18)	0.3244188098818042


## **Partie 4 : Recherche par requête et classement des documents**

### Objectif

L'objectif de cette dernière partie est d'utiliser la **similarité cosinus** pour comparer la requête de l'utilisateur avec les documents de la matrice **TF-IDF**. Les documents seront classés en fonction de leur similarité avec la requête, permettant ainsi de retourner les documents les plus pertinents dans un moteur de recherche.

---

### Consignes

1. Permettre à l'utilisateur d'entrer une requête textuelle (une phrase ou quelques mots).
2. Transformer la requête en vecteur **TF-IDF** en utilisant le même modèle que pour les documents.
3. Calculer la similarité cosinus entre la requête et chaque document du corpus.
4. Classer les documents en fonction de leur score de similarité cosinus.
5. Afficher les documents les plus pertinents avec leur score de similarité, en laissant un espace entre chaque résultat pour faciliter la lecture.

---

In [13]:

from sklearn.metrics.pairwise import cosine_similarity

# Fonction pour effectuer la recherche
def recherche(query, tfidf_matrix, vectorizer):
    # Transformer la requête en vecteur TF-IDF
    query_vector = vectorizer.transform([query])

    # Calculer la similarité cosinus entre la requête et les documents
    similarity = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Trouver les indices des documents les plus similaires
    indices = similarity.argsort()[-5:][::-1]  # Obtenir les 5 documents les plus pertinents

    # Retourner les documents les plus pertinents et leurs scores
    return df.iloc[indices], similarity[indices]

# Exemple de requête
query = "hausse des impôts"
documents_pertinents, scores = recherche(query, tfidf_matrix, vectorizer)

# Afficher chaque document intégralement suivi de son score de similarité
for doc, score in zip(documents_pertinents['Document'], scores):
    print(f"Document :\n{doc}\n")
    print(f"Score de similarité : {score}\n")
    print("-----\n")  # Séparateur entre les documents

Document :
Le succès est partagé et je pense que l’effort doit l’être aussi. Ciblées, exceptionnelles et temporaires, ces hausses ne concerneront donc pas tous les Français. J’aimerais que vous me donniez au moins acte que je n’ai pas parlé d’effort général.

Score de similarité : 0.2177905684815266

-----

Document :
Vous avez évoqué, de même que M. Ciotti, des hausses de la fiscalité. Il convient de dire la vérité. Je répète une fois pour toutes, à l’intention des orateurs des divers groupes, qu’il n’y aura ni choc fiscal ni matraquage. Si nous devons travailler – j’espère le plus longtemps possible – en bonne intelligence ou du moins dans le respect des uns et des autres, je vous demande que vous ne me fassiez pas dire ou que vous ne disiez pas que le Gouvernement fait certaines choses alors que ce n’est pas vrai. Nous allons faire porter l’essentiel de l’effort – vital pour la crédibilité de notre signature, pour celle de notre action et pour la préservation de l’investissement – s